In [1]:
from collections import OrderedDict
import geopandas
import libpysal
from libpysal import cg, examples
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
# import matplotlib_scalebar
# from matplotlib_scalebar.scalebar import ScaleBar
import numpy
# import ortools
# from ortools.linear_solver import pywraplp
# import seaborn
import shapely
from shapely.geometry import Point
import spaghetti
import sys

# n clients and n facilities
client_count, facility_count = 400, 14

# candidate facilites to site
p_facilities = 3

# maximum coverage meters
max_coverage = 1000.0

# minimum coverage meters
min_coverage = 800.0

random_seeds = {"client": 3006, "facility": 1520}

title = "Neighborhood X"

streets = geopandas.read_file(examples.get_path("streets.shp"))
streets.crs = "esri:102649"
streets = streets.to_crs("epsg:2762")
streets.head()

,ID,Length,geometry
0,1.0,244.116229,"LINESTRING (222007.131 267348.711, 222007.159 ..."
1,2.0,375.974828,"LINESTRING (222006.951 267549.880, 222007.131 ..."
2,3.0,400.353405,"LINESTRING (221420.428 267804.889, 221411.402 ..."
3,4.0,660.000000,"LINESTRING (220875.116 268353.388, 220803.948 ..."
4,5.0,660.000000,"LINESTRING (220802.426 268398.824, 220917.000 ..."


In [2]:
def get_buffer(in_data, buff=50):
    """ geopandas.GeoDataFrame should be in a meters projection.
    Parameters
    ----------
    in_data : geopandas.GeoDataFrame
        GeoDataFrame of a shapefile representing a road network.
    buff : int or float
        Desired buffer distance. Default is 50 (meters).
    Returns
    -------
    out_data : geopandas.GeoDataFrame
        Single polygon of the unioned street buffers.
    """
    b1 = in_data.buffer(buff)  # Buffer
    ub = b1.unary_union  # Buffer Union
    b2 = geopandas.GeoSeries(ub)
    out_data = geopandas.GeoDataFrame(b2, crs=in_data.crs, columns=["geometry"])
    return out_data

buff = 50
streets_buffer = get_buffer(streets, buff=buff)
streets_buffer

,geometry
0,"POLYGON ((221824.140 268653.724, 221824.140 26..."


In [3]:
def simulated_geo_points(in_data, needed=20, seed=0, to_file=None):
    """Generate synthetic spatial data points within an area.s
    Parameters
    ----------
    in_data : geopandas.GeoDataFrame
        A single polygon of the unioned street buffers.
    needed : int
        Number of points in the buffer. Default is 20.
    seed : int
        Seed for pseudo-random number generation. Default is 0.
    to_file : str
        File name for write out.
    Returns
    -------
    sim_pts : geopandas.GeoDataFrame
        Points within the buffer.
    """
    geoms = in_data.geometry
    area = tuple(in_data.total_bounds)
    simulated_points_list = []
    simulated_points_all = False
    numpy.random.seed(seed)
    while simulated_points_all == False:
        x = numpy.random.uniform(area[0], area[2], 1)
        y = numpy.random.uniform(area[1], area[3], 1)
        point = Point(x, y)
        if geoms.intersects(point)[0]:
            simulated_points_list.append(point)
        if len(simulated_points_list) == needed:
            simulated_points_all = True
    sim_pts = geopandas.GeoDataFrame(
        simulated_points_list, columns=["geometry"], crs=in_data.crs
    )
    if to_file:
        sim_pts.to_file(to_file + ".shp")
    return sim_pts

clients = simulated_geo_points(
    streets_buffer, needed=client_count, seed=random_seeds["client"]
)
facilities = simulated_geo_points(
    streets_buffer, needed=facility_count, seed=random_seeds["facility"]
)

In [4]:
clients["dv"] = ["x[%s]" % c for c in range(client_count)]
facilities["dv"] = ["y[%s]" % c for c in range(facility_count)]


In [5]:
numpy.random.seed(1991)
clients["weights"] = numpy.random.randint(1, 8, (client_count, 1))
clients.head()

,geometry,dv,weights
0,POINT (220621.917 267350.429),x[0],1
1,POINT (220803.166 268060.603),x[1],7
2,POINT (221870.782 268397.667),x[2],4
3,POINT (220715.998 267148.323),x[3],6
4,POINT (221330.455 267985.572),x[4],5


In [6]:
ntw = spaghetti.Network(in_data=streets)

In [7]:
ntw.snapobservations(clients, "clients", attribute=True)
clients_snapped = spaghetti.element_as_gdf(
    ntw, pp_name="clients", snapped=True
)
ntw.snapobservations(facilities, "facilities", attribute=True)
facilities_snapped = spaghetti.element_as_gdf(
    ntw, pp_name="facilities", snapped=True
)

In [8]:
cost_matrix = ntw.allneighbordistances(
    sourcepattern=ntw.pointpatterns["clients"],
    destpattern=ntw.pointpatterns["facilities"],
)

In [11]:
from spopt.locate.coverage import LSCP
import pulp
lscp = LSCP.from_cost_matrix(cost_matrix, max_coverage)
status = lscp.solve(pulp.CPLEX_CMD())
print(status)

1


In [12]:
lscp.record_decisions()
# print(lscp.cli2iloc)
# print(lscp.cli2fac)
# print(lscp.fac2cli)
print(lscp.n_cli_uncov)
print(lscp.ncov2ncli)

0
{0: 0, 1: 400, 2: 183, 3: 26, 4: 3}


In [20]:
from spopt.locate.coverage import MCLP
import pulp

ai = clients['weights']
ai = ai.values.reshape(cost_matrix.shape[0], 1)

mclp = MCLP.from_cost_matrix(cost_matrix, ai, max_coverage, p_facilities)
status = mclp.solve(pulp.GUROBI_CMD())
print(status)
# for v in model.variables():
#     print(v.name, "=", v.varValue)

1


In [21]:
mclp.record_decisions()
# print(mclp.fac2cli)
# print(mclp.cli2iloc)
# print(mclp.cli2fac)
print(mclp.n_cli_uncov)
print(mclp.ncov2ncli)

6
{0: 6, 1: 394, 2: 99, 3: 17}


In [24]:
import pulp
from spopt.locate.p_median import PMedian

# binary coverage matrix from cij
ai = clients['weights']
ai = ai.values.reshape(cost_matrix.shape[0], 1)

pmedian = PMedian.from_cost_matrix(cost_matrix, ai, p_facilities)
status = pmedian.solve(pulp.CPLEX_CMD())
print(status)

1


In [26]:
import pulp
from spopt.locate.p_center import PCenter

ai = clients['weights']
ai = ai.values.reshape(cost_matrix.shape[0], 1)

pcenter = PCenter.from_cost_matrix(cost_matrix, ai, p_facilities)
status = pcenter.solve(pulp.GUROBI_CMD())
print(status)

1
